In [79]:
#Import libraries
from dash import Dash, dcc, Output, Input  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
import plotly.express as px
import pandas as pd                        # pip install pandas
import panel as pn
import matplotlib.pyplot as plt
pn.extension("tabulator")
import hvplot.pandas #interactive datasets
from panel.interact import interact
pn.extension()
import holoviews as hv
from holoviews import opts
hv.extension("bokeh")
#pn.__version__
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")

In [80]:
# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():
    # incorporate data into app, this Alloy_Mat come from the whole matriz being interpolated and Material added, but then again I should perhaps use Alloy_Null
    Alloy_Null= pd.read_csv("../data/Alloy_Null.csv")   
    pn.state.cache['data'] = Alloy_Null.copy()
else:
    Alloy_Null=pn.state.cache["data"]

In [37]:
#Data is already preprocessed, just round
Alloy_Null= Alloy_Null.round(2)
Alloy_Null.head()

,Cl,Test_Temp,pH,M_Al,M_Fe,M_HEA,M_NiCrMo,M_Other,Fe,Cr,...,Re,Ce,Ti,Co,B,Mg,Y,Gd,Epit,Material
0,0.01,50.0,7.8,0.0,1.0,0.0,0.0,0.0,69.7,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,287.0,Fe Alloy
1,0.01,50.0,7.8,0.0,1.0,0.0,0.0,0.0,69.7,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,260.0,Fe Alloy
2,0.02,50.0,7.8,0.0,1.0,0.0,0.0,0.0,69.7,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,218.0,Fe Alloy
3,0.09,50.0,7.8,0.0,1.0,0.0,0.0,0.0,69.7,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,140.0,Fe Alloy
4,0.55,50.0,8.2,0.0,1.0,0.0,0.0,0.0,69.7,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,94.0,Fe Alloy


In [62]:
Alloy_Null[Alloy_Null["M_HEA"]==1].shape

(27, 30)

In [64]:
#Make DataFrame pipeline Interactive
iAlloys=Alloy_Null.interactive()

In [65]:
#Define Panel Widgets
Cr_slider=pn.widgets.IntSlider(name="Cr Slider", start=0, end=100)

In [66]:
#Radio button for Corrosion Measures
yaxis_Epit=pn.widgets.RadioButtonGroup(
    name="Y axis",
    options=["Epit"],
    button_type="success"
)

In [67]:
Materials=['Fe Alloy', 'Other', 'HEA', 'NiCrMo Alloy', 'Al Alloy']

In [74]:
Alloy_pipeline=(
    iAlloys[
        (iAlloys.Cr <= Cr_slider)&
        (iAlloys.Material.isin(Materials))
    ]
    .groupby(["Material", "Cr"])[yaxis_Epit].mean()
    .to_frame()
    .reset_index()
    .sort_values(by="Cr")
    .reset_index(drop=True)
)

In [94]:
Alloy_plot= Alloy_pipeline.hvplot(x="Cr", by = "Material", y=yaxis_Epit, kind="scatter", size=80, alpha=0.7, height= 500, width =600,
                    title = "Pitting Corrosion Potential by varying Cr for different Material Types")
Alloy_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'9bed91b6-8b6f-4b9b-926f-8a219c2331ab': {'defs': …

### Corrosion Pitting Potential Epit vs Chloride
Filter the Data per Material type

In [95]:
#Define Panel Widgets
Cl_slider=pn.widgets.IntSlider(name="Cl_Slider", start=0, end=10)

In [97]:
#Radio button for Corrosion Measures
yaxis_E_Cl =pn.widgets.RadioButtonGroup(
    name="Y Cl_axis",
    options=["Epit"],
    button_type="success"
)
Materials=['Fe Alloy', 'Other', 'HEA', 'NiCrMo Alloy', 'Al Alloy']

In [98]:
Epit_vs_Cl_scatterplot_pipeline=(
    iAlloys[
        (iAlloys.Cl <= Cl_slider) &
        (iAlloys.Material.isin(Materials))
    ]
    .groupby(["Material", "Cl"])["Epit"].mean()
    .to_frame()
    .reset_index()
    .sort_values(by="Epit")
    .reset_index(drop=True)
)

In [99]:
Epit_vs_Cl_scatterplot= Epit_vs_Cl_scatterplot_pipeline.hvplot(x="Cl", y="Epit", by= "Material", size=80, kind="scatter", alpha=0.7, legend=True, height= 600, width =500)

### Dashboard Creation

In [100]:
#Layout using Template
template =pn.template.FastListTemplate(
    title= "Corrosion Prediction Dashboard",
    sidebar =[pn.pane.Markdown("# Pitting Corrosion influencers"),
              pn.pane.Markdown("#### Pitting Corrosion is a primary driver of corrosion in your operational systems that should be monitored to know the risk so to be able to prevent future problems and mitigate the damage"),
              pn.pane.JPG("../images/Corr_simbol.jpg", sizing_mode="scale_both"),
              pn.pane.Markdown("## Settings"),
              Cr_slider],
    main=[pn.Row(pn.Column(yaxis_Epit, 
                           Alloy_plot.panel(width=700), margin=(0,25))),
          pn.Row(pn.Column(Epit_vs_Cl_scatterplot.panel(width=600), margin=(0,25)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)

In [93]:
template.show()
template.servable();

Launching server at http://localhost:52381


### Pitting Corrosion Measure by Circular Polarimetry (Electrochemical Potential vs Current density)

In [50]:
# incorporate data into app
Pitt= pd.read_csv("../data/pitting.csv")
#Make DataFrame pipeline Interactive
iPitt= Pitt.interactive()
#Define Panel Widgets
Potential_slider=pn.widgets.IntSlider(name="E Potential Slider", start=-1000, end=-400)
#Radio button for pitting corrosion measures
#Creating a new y
yaxis_pitting=pn.widgets.RadioButtonGroup(
    name="Y axis_p",
    options=["Potential"],
    button_type="success"
)

Materials=['Fe Alloy', 'Other', 'HEA', 'NiCrMo Alloy', 'Al Alloy']
Pitting_pipeline=(
    iPitt[
        (iPitt.Potential <= Potential_slider)
    ]
    .groupby(["Current_density"])[yaxis_pitting].mean()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)
Pitting_plot= Pitting_pipeline.hvplot(x="Potential", y="Current_density", kind="scatter", size=80, alpha=0.7, height= 500, width =500,
                    title = "Pitting Corrosion Potential by varying Cr for different Material Types")
Pitting_plot

ERROR:tornado.application:Uncaught exception GET / (::1)
HTTPServerRequest(protocol='http', host='localhost:60626', method='GET', uri='/', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "/Users/beatrizamandawatts/neufische/Corrosion_85_prediction/.venv/lib/python3.9/site-packages/tornado/web.py", line 1713, in _execute
    result = await result
  File "/Users/beatrizamandawatts/neufische/Corrosion_85_prediction/.venv/lib/python3.9/site-packages/panel/io/server.py", line 337, in get
    session = await self.get_session()
  File "/Users/beatrizamandawatts/neufische/Corrosion_85_prediction/.venv/lib/python3.9/site-packages/bokeh/server/views/session_handler.py", line 144, in get_session
    session = await self.application_context.create_session_if_needed(session_id, self.request, token)
  File "/Users/beatrizamandawatts/neufische/Corrosion_85_prediction/.venv/lib/python3.9/site-packages/bokeh/server/contexts.py", line 243, in create_session_if_needed
    s

BokehModel(combine_events=True, render_bundle={'docs_json': {'24a4d1fd-93b0-4e88-8faf-387959dfed8a': {'defs': …

### Epit by Material Type using different technique

Using iAlloys data

In [51]:
Alloys=Alloy_Null
Features=['Material', 'Cl', 'Test_Temp', 'pH', 'Fe', 'Cr', 'V', 'Ni', 'Mo', 'W', 'Nb',
          'Al',  'Ti', 'Co', 'B', 'Mg', 'Y', 'Gd', 'Epit']

Alloys_x= pn.widgets.Select(name="x", options= Features, value="Epit" )
Alloys_y= pn.widgets.Select(name="y", options= Features)

dashboard_fit_line_checkbox=pn.widgets.Checkbox(name="fit Line")
Alloys_data= pn.widgets.Select(name="Features", options=Features)

#Create functions using depends decorator
@pn.depends(Alloys_x, Alloys_y, dashboard_fit_line_checkbox)
def create_scater_plots(x, y, checkbox):
    Alloys_scatter= Alloys.hvplot.scatter(x, y).opts(width=450)
    if checkbox == True:
        return Alloys_scatter * hv.Slope.from_scatter(Alloys_scatter)
    else:
        return Alloys_scatter
###@pn.depends(Alloys_data)
#def create_boxplot(Features):
  ##return Alloys.hvplot.box(y=Features).opts(width=450)


#layout
pn.Row(
    pn.Column("## Factors that influence Corrosion Pitting",
              pn.Column(Alloys_x, Alloys_y, dashboard_fit_line_checkbox),
              create_scater_plots),
   ''' pn.Spacer(width=50),
    pn.Column("## Boxplot",
    "## Min, 23%, Median, 75%, Max, Outliers",
    Alloys_data,
    create_boxplot)'''

SyntaxError: unexpected EOF while parsing (2204019001.py, line 33)